# Source Intake Agent: VCF File Processing

## Overview

This notebook implements the **Source Intake Agent** - the first agent in our multi-source WES pipeline automation system.

### Agent Purpose
- **Standardize input**: Handle VCF files from different sources (Mayo Tapestry/Helix, Mayo BioBank/Regeneron, UK Biobank)
- **Extract metadata**: Identify source, sample counts, reference build
- **Validate integrity**: Check file format, corruption, required fields
- **Route processing**: Tag files for source-specific downstream processing

### Architecture
```
┌─────────────────────────────────────────┐
│      Source Intake Agent                │
│  - identify_source()                    │
│  - validate_vcf_integrity()             │
│  - extract_source_metadata()            │
└─────────────────────────────────────────┘
           ↓
    bcftools, subprocess
```

### Test Data
- **toy.vcf**: 10,000 lines from 1000 Genomes Chr22 (samples: NA12878, NA12889, NA12890)
- Source: 1000 Genomes Phase 3
- Build: GRCh37/hg19

## Setup and Imports

In [14]:
import os
import subprocess
import json
from typing import Dict, List, Any, Literal
from pathlib import Path
import pandas as pd
import re
from dotenv import load_dotenv

# Google ADK imports
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner

# Load environment variables from .env file
load_dotenv()

# Set up paths
DATA_DIR = Path("data")
TOY_VCF = DATA_DIR / "toy.vcf"

# API key (if not already set)
if "GOOGLE_API_KEY" not in os.environ:
    raise ValueError(
        "GOOGLE_API_KEY not found in environment. Please set it in .env file."
    )

# Model selection
LLM_MODEL = "gemini-2.5-flash-lite"

print(f"✓ Setup complete")
print(f"✓ Using model: {LLM_MODEL}")
print(f"✓ Test VCF: {TOY_VCF}")
print(f"✓ API key loaded from .env")

✓ Setup complete
✓ Using model: gemini-2.5-flash-lite
✓ Test VCF: data/toy.vcf
✓ API key loaded from .env


## Tool 1: Identify Source

Analyzes VCF header and filename to identify data source.

In [5]:
def identify_source(vcf_path: str) -> dict:
    """
    Identify VCF source from metadata and filename.

    Args:
        vcf_path: Path to VCF file (can be .vcf or .vcf.gz)

    Returns:
        Dictionary with source, n_samples, n_variants, build, and other metadata
    """
    vcf_path = Path(vcf_path)

    if not vcf_path.exists():
        return {"error": "File not found", "vcf_path": str(vcf_path)}

    # Determine if gzipped
    is_gzipped = vcf_path.suffix == ".gz"

    # Extract header
    if is_gzipped:
        result = subprocess.run(
            ["bcftools", "view", "-h", str(vcf_path)], capture_output=True, text=True
        )
    else:
        result = subprocess.run(
            ["grep", "^#", str(vcf_path)], capture_output=True, text=True
        )

    if result.returncode != 0:
        return {"error": "Failed to read VCF header", "stderr": result.stderr}

    header = result.stdout

    # Parse metadata from header
    source = "unknown"
    vcf_name = vcf_path.name.lower()

    # Detect source from header and filename
    if "helix" in header.lower() or "tapestry" in vcf_name:
        source = "mayo_tapestry_helix"
    elif "regeneron" in header.lower() or "biobank" in vcf_name:
        source = "mayo_biobank_regeneron"
    elif "ukbiobank" in vcf_name or "ukb" in vcf_name:
        source = "ukbiobank"
    elif "1000genomes" in header.lower() or "1kg" in vcf_name or "phase3" in vcf_name:
        source = "1000genomes_phase3"

    # Count samples from header line
    header_lines = header.strip().split("\n")
    chrom_line = [l for l in header_lines if l.startswith("#CHROM")]

    n_samples = 0
    sample_ids = []
    if chrom_line:
        fields = chrom_line[0].split("\t")
        # Samples start after FORMAT column (index 9)
        sample_ids = fields[9:] if len(fields) > 9 else []
        n_samples = len(sample_ids)

    # Count variants (excluding headers)
    if is_gzipped:
        count_result = subprocess.run(
            f"bcftools view -H {vcf_path} | wc -l",
            shell=True,
            capture_output=True,
            text=True,
        )
    else:
        count_result = subprocess.run(
            f"grep -v '^#' {vcf_path} | wc -l",
            shell=True,
            capture_output=True,
            text=True,
        )

    n_variants = int(count_result.stdout.strip()) if count_result.returncode == 0 else 0

    # Detect reference build
    build = "unknown"
    if "GRCh38" in header or "hg38" in header:
        build = "GRCh38"
    elif "GRCh37" in header or "hg19" in header or "b37" in header:
        build = "GRCh37"

    # Extract fileformat version
    format_line = [l for l in header_lines if l.startswith("##fileformat=")]
    vcf_version = format_line[0].split("=")[1] if format_line else "unknown"

    # Extract source line if present
    source_line = [l for l in header_lines if l.startswith("##source=")]
    source_info = source_line[0].split("=")[1] if source_line else "not specified"

    return {
        "source": source,
        "n_samples": n_samples,
        "sample_ids": sample_ids,
        "n_variants": n_variants,
        "build": build,
        "vcf_version": vcf_version,
        "source_info": source_info,
        "vcf_path": str(vcf_path),
        "is_gzipped": is_gzipped,
    }


# Test the function directly
print("Testing identify_source() on toy.vcf...")
result = identify_source(str(TOY_VCF))
print(json.dumps(result, indent=2))

Testing identify_source() on toy.vcf...
{
  "source": "1000genomes_phase3",
  "n_samples": 3,
  "sample_ids": [
    "NA12878",
    "NA12889",
    "NA12890"
  ],
  "n_variants": 9745,
  "build": "GRCh37",
  "vcf_version": "VCFv4.1",
  "source_info": "1000GenomesPhase3Pipeline",
  "vcf_path": "data/toy.vcf",
  "is_gzipped": false
}


## Tool 2: Validate VCF Integrity

Checks file format compliance, required fields, and basic integrity.

In [6]:
def validate_vcf_integrity(vcf_path: str) -> dict:
    """
    Check VCF file integrity (format compliance, required fields).

    Args:
        vcf_path: Path to VCF file

    Returns:
        Dictionary with validation checks and overall pass/fail
    """
    vcf_path = Path(vcf_path)
    checks = {}

    # 1. Check if file exists
    checks["file_exists"] = vcf_path.exists()
    if not checks["file_exists"]:
        checks["passed"] = False
        checks["error"] = "File not found"
        return checks

    # 2. Check if indexed (for .gz files)
    is_gzipped = vcf_path.suffix == ".gz"
    if is_gzipped:
        checks["has_index"] = (vcf_path.parent / f"{vcf_path.name}.tbi").exists() or (
            vcf_path.parent / f"{vcf_path.name}.csi"
        ).exists()
    else:
        checks["has_index"] = None  # Not applicable for uncompressed

    # 3. Check format compliance - try to read header
    try:
        if is_gzipped:
            result = subprocess.run(
                ["bcftools", "view", "-h", str(vcf_path)],
                capture_output=True,
                text=True,
                check=True,
                timeout=30,
            )
        else:
            result = subprocess.run(
                ["grep", "^#", str(vcf_path)],
                capture_output=True,
                text=True,
                check=True,
                timeout=30,
            )
        checks["format_valid"] = True
        header = result.stdout
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired) as e:
        checks["format_valid"] = False
        checks["error"] = f"VCF format invalid or file corrupted: {str(e)}"
        checks["passed"] = False
        return checks

    # 4. Check required header fields
    required_fields = ["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]
    chrom_line = [l for l in header.split("\n") if l.startswith("#CHROM")]
    if chrom_line:
        header_fields = chrom_line[0].split("\t")
        checks["has_required_fields"] = all(f in header_fields for f in required_fields)
    else:
        checks["has_required_fields"] = False

    # 5. Check fileformat declaration
    checks["has_fileformat"] = "##fileformat=VCF" in header

    # 6. Sample a few variants to check data integrity
    try:
        if is_gzipped:
            test = subprocess.run(
                ["bcftools", "view", "-H", str(vcf_path), "|", "head", "-100"],
                shell=True,
                capture_output=True,
                text=True,
                check=True,
                timeout=30,
            )
        else:
            test = subprocess.run(
                f"grep -v '^#' {vcf_path} | head -100",
                shell=True,
                capture_output=True,
                text=True,
                check=True,
                timeout=30,
            )

        sample_lines = [l for l in test.stdout.strip().split("\n") if l]
        checks["sample_valid"] = len(sample_lines) > 0
        checks["sample_lines_count"] = len(sample_lines)

        # Check that lines have expected number of fields
        if sample_lines and chrom_line:
            expected_fields = len(header_fields)
            actual_fields = len(sample_lines[0].split("\t"))
            checks["field_count_match"] = actual_fields == expected_fields

    except (subprocess.CalledProcessError, subprocess.TimeoutExpired):
        checks["sample_valid"] = False
        checks["field_count_match"] = False

    # Overall pass/fail
    required_checks = [
        checks["format_valid"],
        checks["has_required_fields"],
        checks["has_fileformat"],
        checks["sample_valid"],
    ]

    checks["passed"] = all(required_checks)

    return checks


# Test the function
print("Testing validate_vcf_integrity() on toy.vcf...")
result = validate_vcf_integrity(str(TOY_VCF))
print(json.dumps(result, indent=2))

Testing validate_vcf_integrity() on toy.vcf...
{
  "file_exists": true,
  "has_index": null,
  "format_valid": true,
  "has_required_fields": true,
  "has_fileformat": true,
  "sample_valid": true,
  "sample_lines_count": 100,
  "field_count_match": true,
  "passed": true
}


## Tool 3: Extract Source-Specific Metadata

Extracts detailed metadata for downstream processing based on identified source.

In [7]:
def extract_source_metadata(vcf_path: str, source: str) -> dict:
    """
    Extract source-specific metadata for downstream normalization.

    Args:
        vcf_path: Path to VCF file
        source: Identified source (from identify_source)

    Returns:
        Dictionary with source-specific processing parameters and expectations
    """
    vcf_path = Path(vcf_path)

    # Source-specific processing parameters
    source_configs = {
        "mayo_tapestry_helix": {
            "capture_kit": "helix_exome_v2",
            "expected_depth_range": (20, 30),
            "expected_ti_tv": (2.8, 3.2),
            "platform": "Illumina NovaSeq",
            "population": "Mayo patient population (Midwest US, European ancestry enriched)",
            "qc_thresholds": {
                "min_depth": 10,
                "min_qual": 30,
                "max_missing_rate": 0.05,
            },
        },
        "mayo_biobank_regeneron": {
            "capture_kit": "regeneron_exome",
            "expected_depth_range": (30, 40),
            "expected_ti_tv": (2.8, 3.2),
            "platform": "Illumina sequencing (RGC)",
            "population": "Mayo BioBank",
            "qc_thresholds": {
                "min_depth": 15,
                "min_qual": 30,
                "max_missing_rate": 0.03,
            },
        },
        "ukbiobank": {
            "capture_kit": "mixed_illumina_idt",
            "expected_depth_range": (15, 25),
            "expected_ti_tv": (2.7, 3.3),
            "platform": "Multiple vendors (Illumina HiSeq, NovaSeq)",
            "population": "UK population (diverse)",
            "qc_thresholds": {"min_depth": 8, "min_qual": 20, "max_missing_rate": 0.10},
        },
        "1000genomes_phase3": {
            "capture_kit": "whole_genome",
            "expected_depth_range": (5, 10),
            "expected_ti_tv": (2.0, 2.1),  # Whole genome has different Ti/Tv
            "platform": "Illumina (multiple)",
            "population": "Global diversity (26 populations)",
            "qc_thresholds": {"min_depth": 4, "min_qual": 20, "max_missing_rate": 0.15},
        },
        "unknown": {
            "capture_kit": "unknown",
            "expected_depth_range": (10, 50),
            "expected_ti_tv": (2.0, 3.5),
            "platform": "unknown",
            "population": "unknown",
            "qc_thresholds": {"min_depth": 5, "min_qual": 20, "max_missing_rate": 0.10},
        },
    }

    config = source_configs.get(source, source_configs["unknown"])

    # Get basic file info
    file_size_mb = vcf_path.stat().st_size / (1024 * 1024)

    # Get chromosome coverage
    is_gzipped = vcf_path.suffix == ".gz"
    if is_gzipped:
        chr_result = subprocess.run(
            f"bcftools view -H {vcf_path} | cut -f1 | sort -u",
            shell=True,
            capture_output=True,
            text=True,
        )
    else:
        chr_result = subprocess.run(
            f"grep -v '^#' {vcf_path} | cut -f1 | sort -u",
            shell=True,
            capture_output=True,
            text=True,
        )

    chromosomes = (
        chr_result.stdout.strip().split("\n") if chr_result.returncode == 0 else []
    )

    return {
        "source": source,
        "capture_kit": config["capture_kit"],
        "expected_depth_range": config["expected_depth_range"],
        "expected_ti_tv": config["expected_ti_tv"],
        "platform": config["platform"],
        "population": config["population"],
        "qc_thresholds": config["qc_thresholds"],
        "file_size_mb": round(file_size_mb, 2),
        "chromosomes": chromosomes,
        "n_chromosomes": len(chromosomes),
    }


# Test the function
print("Testing extract_source_metadata() on toy.vcf...")
result = extract_source_metadata(str(TOY_VCF), "1000genomes_phase3")
print(json.dumps(result, indent=2))

Testing extract_source_metadata() on toy.vcf...
{
  "source": "1000genomes_phase3",
  "capture_kit": "whole_genome",
  "expected_depth_range": [
    5,
    10
  ],
  "expected_ti_tv": [
    2.0,
    2.1
  ],
  "platform": "Illumina (multiple)",
  "population": "Global diversity (26 populations)",
  "qc_thresholds": {
    "min_depth": 4,
    "min_qual": 20,
    "max_missing_rate": 0.15
  },
  "file_size_mb": 1.42,
  "chromosomes": [
    "22"
  ],
  "n_chromosomes": 1
}


## Create the Source Intake Agent

Now we'll create the agent that orchestrates these tools.

In [8]:
intake_agent = Agent(
    name="VCF_Source_Intake_Agent",
    model=LLM_MODEL,
    instruction="""
    You are a VCF Source Intake Specialist for genomics pipelines.
    
    YOUR MISSION:
    Process incoming VCF files and prepare them for downstream analysis by:
    1. Identifying the data source (Mayo Tapestry/Helix, Mayo BioBank/Regeneron, UK Biobank, 1000 Genomes, etc.)
    2. Validating file integrity and format compliance
    3. Extracting source-specific metadata for pipeline configuration
    4. Flagging any quality concerns or anomalies
    
    TOOLS AVAILABLE:
    - identify_source(vcf_path): Identifies data source, counts samples/variants, detects reference build
    - validate_vcf_integrity(vcf_path): Checks file format, required fields, basic integrity
    - extract_source_metadata(vcf_path, source): Gets source-specific processing parameters
    
    WORKFLOW:
    1. Always run identify_source() FIRST
    2. Then run validate_vcf_integrity()
    3. If validation passes, run extract_source_metadata() with the identified source
    4. Summarize findings and flag any concerns
    
    VALIDATION GATES (automatic rejection):
    - VCF format invalid
    - Missing required fields
    - No samples in VCF
    - Cannot read file
    
    HUMAN REVIEW TRIGGERS (flag but don't reject):
    - Unknown source detected
    - Sample count unusually low (< 10) or high (> 100,000)
    - Reference build mismatch with expected
    - Missing index file for .gz
    
    COMMUNICATION STYLE:
    - Be systematic and thorough
    - Report findings clearly with specific numbers
    - Use sections: Source Identification, Validation Results, Metadata Summary, Recommendations
    - Flag concerns with ⚠️ and approvals with ✓
    - If validation fails, STOP and report - do NOT proceed to metadata extraction
    """,
    tools=[identify_source, validate_vcf_integrity, extract_source_metadata],
)

print("✓ Source Intake Agent created")

✓ Source Intake Agent created


## Test the Agent on toy.vcf

Let's run the complete intake process on our test file.

In [9]:
# Create runner
runner = InMemoryRunner(agent=intake_agent)

# Test query
query = f"""
Please process this VCF file for intake into our genomics pipeline:

File: {TOY_VCF}

Execute the full intake workflow:
1. Identify the data source
2. Validate file integrity  
3. Extract source-specific metadata
4. Provide recommendations for downstream processing

Be thorough and flag any concerns.
"""

print("🧬 Starting VCF Source Intake Process")
print("=" * 80)
print(f"Target file: {TOY_VCF}")
print("=" * 80 + "\n")

# Run the agent
response = await runner.run_debug(query)

print("\n" + "=" * 80)
print("✅ INTAKE PROCESS COMPLETE")
print("=" * 80)

🧬 Starting VCF Source Intake Process
Target file: data/toy.vcf


 ### Created new session: debug_session_id

User > 
Please process this VCF file for intake into our genomics pipeline:

File: data/toy.vcf

Execute the full intake workflow:
1. Identify the data source
2. Validate file integrity  
3. Extract source-specific metadata
4. Provide recommendations for downstream processing

Be thorough and flag any concerns.



VCF_Source_Intake_Agent > Okay, I will start by identifying the source of the VCF file.

Source Identification:
*   **Source:** 1000genomes_phase3
*   **Sample Count:** 3
*   **Variant Count:** 9745
*   **Reference Build:** GRCh37
*   **VCF Version:** VCFv4.1
*   **Sample IDs:** NA12878, NA12889, NA12890
*   **Source Info:** 1000GenomesPhase3Pipeline

Proceeding to validation.
VCF_Source_Intake_Agent > Validation Results:
The VCF file `data/toy.vcf` is valid in format and contains the required fields. All checks passed. ✓

Now, I will extract source-specific metadata.
VCF_Source_Intake_Agent > Metadata Summary:
*   **Source:** 1000genomes_phase3
*   **Capture Kit:** whole_genome
*   **Platform:** Illumina (multiple)
*   **Population:** Global diversity (26 populations)
*   **Chromosomes:** 22 (contains only chromosome 22)
*   **Expected Depth Range:** 5-10x
*   **Expected Ti/Tv Ratio:** 2.0-2.1
*   **File Size:** 1.42 MB
*   **QC Thresholds:** Max Missing Rate: 0.15, Min Depth: 4, Min 

## Manual Inspection of VCF File

Let's also manually inspect the file to understand its structure.

In [10]:
# Display header lines
print("VCF Header (first 50 lines):")
print("=" * 80)
with open(TOY_VCF, "r") as f:
    header_count = 0
    for line in f:
        if line.startswith("#"):
            print(line.rstrip())
            header_count += 1
            if header_count >= 50:
                break

print(f"\n(Showing first {header_count} header lines)")

VCF Header (first 50 lines):
##fileformat=VCFv4.1
##FILTER=<ID=PASS,Description="All filters passed">
##fileDate=20150218
##reference=ftp://ftp.1000genomes.ebi.ac.uk//vol1/ftp/technical/reference/phase2_reference_assembly_sequence/hs37d5.fa.gz
##source=1000GenomesPhase3Pipeline
##contig=<ID=1,assembly=b37,length=249250621>
##contig=<ID=2,assembly=b37,length=243199373>
##contig=<ID=3,assembly=b37,length=198022430>
##contig=<ID=4,assembly=b37,length=191154276>
##contig=<ID=5,assembly=b37,length=180915260>
##contig=<ID=6,assembly=b37,length=171115067>
##contig=<ID=7,assembly=b37,length=159138663>
##contig=<ID=8,assembly=b37,length=146364022>
##contig=<ID=9,assembly=b37,length=141213431>
##contig=<ID=10,assembly=b37,length=135534747>
##contig=<ID=11,assembly=b37,length=135006516>
##contig=<ID=12,assembly=b37,length=133851895>
##contig=<ID=13,assembly=b37,length=115169878>
##contig=<ID=14,assembly=b37,length=107349540>
##contig=<ID=15,assembly=b37,length=102531392>
##contig=<ID=16,assembly=

In [11]:
# Display first few data lines
print("\nFirst 5 variant records:")
print("=" * 80)
with open(TOY_VCF, "r") as f:
    data_count = 0
    for line in f:
        if not line.startswith("#"):
            # Truncate long lines for readability
            if len(line) > 200:
                print(line[:200].rstrip() + "...")
            else:
                print(line.rstrip())
            data_count += 1
            if data_count >= 5:
                break


First 5 variant records:
22	16050075	.	A	G	100	PASS	AC=0;AF=0.000199681;AN=6;NS=2504;DP=8012;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0;SAS_AF=0.001;AA=.|||;VT=SNP	GT	0|0	0|0	0|0
22	16050115	.	G	A	100	PASS	AC=0;AF=0.00638978;AN=6;NS=2504;DP=11468;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0.0234;EUR_AF=0;SAS_AF=0;AA=.|||;VT=SNP	GT	0|0	0|0	0|0
22	16050213	.	C	T	100	PASS	AC=0;AF=0.00758786;AN=6;NS=2504;DP=15092;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0.0272;EUR_AF=0.001;SAS_AF=0;AA=.|||;VT=SNP	GT	0|0	0|0	0|0
22	16050319	.	C	T	100	PASS	AC=0;AF=0.000199681;AN=6;NS=2504;DP=22609;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0;EUR_AF=0;SAS_AF=0;AA=.|||;VT=SNP	GT	0|0	0|0	0|0
22	16050527	.	C	A	100	PASS	AC=0;AF=0.000199681;AN=6;NS=2504;DP=23591;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0.001;SAS_AF=0;AA=.|||;VT=SNP	GT	0|0	0|0	0|0


## Summary Statistics

Let's get some basic statistics about the file.

In [12]:
# Basic statistics
def get_vcf_stats(vcf_path):
    stats = {}

    with open(vcf_path, "r") as f:
        header_lines = 0
        data_lines = 0
        sample_line = None

        for line in f:
            if line.startswith("##"):
                header_lines += 1
            elif line.startswith("#CHROM"):
                sample_line = line
                header_lines += 1
            else:
                data_lines += 1

        stats["header_lines"] = header_lines
        stats["data_lines"] = data_lines

        if sample_line:
            fields = sample_line.strip().split("\t")
            stats["samples"] = fields[9:]
            stats["n_samples"] = len(fields[9:])

    return stats


stats = get_vcf_stats(TOY_VCF)

print("VCF File Statistics:")
print("=" * 80)
print(f"Total header lines: {stats['header_lines']}")
print(f"Total variant lines: {stats['data_lines']}")
print(f"Number of samples: {stats['n_samples']}")
print(f"Sample IDs: {', '.join(stats['samples'])}")
print(f"File size: {TOY_VCF.stat().st_size / (1024*1024):.2f} MB")

VCF File Statistics:
Total header lines: 255
Total variant lines: 9745
Number of samples: 3
Sample IDs: NA12878, NA12889, NA12890
File size: 1.42 MB


## Next Steps

Now that we have the Source Intake Agent working, the next agents to implement are:

1. **QC/Validation Agent** (Agent 2)
   - Sample-level QC (missingness, heterozygosity, Ti/Tv)
   - Variant-level QC (missing rate, HWE, MAF)
   - Sex concordance checks
   - Generate QC reports

2. **Harmonization Agent** (Agent 3)
   - Normalize variant representation
   - Handle different exome capture regions
   - Strip to minimal fields

3. **Batch Normalization Agent** (Agent 4)
   - Detect batch effects via PCA
   - Apply ComBat correction if needed
   - Preserve population structure

4. **H5 Conversion Agent** (Agent 5)
   - Convert to PyTorch-ready format
   - Validate H5 integrity
   - Generate DataLoader example

5. **Pipeline Supervisor**
   - Orchestrate all agents
   - Handle QC gate checkpoints
   - Track lineage and provenance